# Gai/Gen: Text-to-Text (TTT)

## 1.1 Setting Up

We will create a seperate virtual environment for this to avoid conflicting dependencies that each underlying model requires.

```sh
sudo apt update -y && sudo apt install ffmpeg git git-lfs -y
conda create -n TTT python=3.10.10 -y
conda activate TTT
pip install -e ".[TTT]"
```

The following examples has been tested on the following environment:

-   NVidia GeForce RTX 2060 6GB
-   Windows 11 + WSL2
-   Ubuntu 22.04
-   Python 3.10
-   CUDA Toolkit 11.8
-   openai 1.6.1
-   anthropic 0.8.1
-   transformers 4.36.2
-   bitsandbytes 0.41.3.post2
-   scipy 1.11.4
-   accelerate 0.25.0
-   llama-cpp-python 0.2.25


## 1.2 Running as a Library

### OpenAI GPT4

For (1) and (2) below, you will use the GaiGen library to call OpenAI's GPT4.
You will need to get an API key from OpenAI. 
Create .env file in project root directory and insert the OpenAI API Key below:

```sh
OPENAI_API_KEY=<your key here>
```

In [ ]:
### 1. GPT4 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}], max_tokens=100,stream=False)
print(response.choices[0].message.content)

In [ ]:
### 2. GPT4 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],stream=True)
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='',flush=True)

### Mistral 7B 8k-context 4-bit quantized

For (3) and (4), you will run Mistral 7B locally. Clone TheBloke's 4-bit quantized version of Mistral-7B model from hugging face. This model utilizes the exLlama loader for increased performance. Make sure you have huggingface-hub installed, if not run `pip install huggingface-hub`.

In [ ]:
%%bash
huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GPTQ \
        config.json
        model.safetensors 
        tokenizer.model
        --local-dir ~/gai/models/Mistral-7B-Instruct-v0.1-GPTQ \
        --local-dir-use-symlinks False

In [ ]:
### 3. Mistral Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100, stream=False)
print(response.choices[0].message.content)


In [ ]:
### 4. Mistral Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    if (chunk.choices[0].delta.content):
        print(chunk.choices[0].delta.content,end='',flush=True)


### Yarn-Mistral-7B 128k-context 4-bit quantized

Repeat the earlier examples but using a different version of Mistral-7B model with a larger context window.

In [ ]:
%%bash
huggingface-cli download TheBloke/Yarn-Mistral-7B-128k-GPTQ \
        --local-dir ~/gai/models/Yarn-Mistral-7B-128k-GPTQ \
        --local-dir-use-symlinks False

According to their paper, the perplexity seems better than the original once the token length is greater than 10k.

![perplexity-of-mistral7b-128k](https://raw.githubusercontent.com/jquesnelle/yarn/mistral/data/proofpile-long-small-mistral.csv.png)



In [ ]:
### 3. Mistral Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b_128k-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 4. Mistral Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b_128k-exllama')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    if (chunk.choices[0].delta.content):
        print(chunk.choices[0].delta.content,end='',flush=True)

### Anthropics Claude2.1

The following example uses Anthropics Claude2.1 100k context window size model. Get API Key from Anthropics and add it to the .env file.
```sh
ANTHROPIC_APIKEY=<your key here>
```

In [ ]:
### 5. Claude-2.1 Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 6. Claude-2.1 Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('claude2-100k')
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_tokens_to_sample=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

### Llama2 7B with HuggingFace transformers

Follow the instructions [here](https://huggingface.co/docs/transformers/main/en/model_doc/llama2) to signup with Meta to download the LLaMa-2 model.
Download the model in HuggingFace format from [here] (https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) into ~/gai/models/Llama-2-7b-chat-hf.

In [ ]:
### 7. Llama2-7B Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
from IPython.utils import io
with io.capture_output() as captured:
    gen = Gaigen.GetInstance().load('llama2-transformers')
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
### 8. Llama2-7B Text-to-Text Streaming

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('llama2-transformers')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

### Llama2 7B GGUF with LlaMaCPP (CPU only)

The following example uses GGUF formatted version of Mistral-7B for LlaMaCPP. This can be used when you want the model to run off CPU only

In [ ]:
%%bash
# Download the model
huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF \
                mistral-7b-instruct-v0.1.Q4_K_M.gguf  \
                config.json \
                --local-dir ~/gai/models/Mistral-7B-Instruct-v0.1-GGUF \
                --local-dir-use-symlinks False

In [ ]:
## 9. Mistral-7B CPU-Only Text-to-Text Generation

print("GENERATING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
import sys
with io.capture_output() as captured:
    # Redirect stderr to stdout
    sys.stderr = sys.stdout    
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100)
print(response.choices[0].message.content,end='',flush=True)

In [ ]:
## 10. Mistral-7B CPU-Only Text-to-Text Generation

print("STREAMING:")
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-llamacpp')
from IPython.utils import io
with io.capture_output() as captured:
    response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],max_new_tokens=100,stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content,end='',flush=True)

## 1.3 Using Function Call

OpenAPI provided a powerful feature for its API called Function calling. It is essentially a way for the LLM to seek external help when encountering limitation to its ability to generate text but returning a string emulating the calling of a function based on the function description provied by the user.

We will create a set of tools that can be made available to the models below.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "gg",
            "description": "The 'gg' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "scrape",
            "description": "Scrape the content of the provided url",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url to scrape the content from"
                    }
                },
                "required": ["url"]
            }
        }
    }
]

Next we will show how it works in gpt-4.

In [4]:
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')

print("Model decided to use tool: ")
user_prompt = "Where did PM Lee Hsien Loong hold his 2023 national day rally?"
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
        tools=tools,
        stream=False,
        max_new_tokens=200)
print(response.choices[0].message)

print("Model decided not to use tool: ")
user_prompt = "Tell me a one paragraph story."
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
        tools=tools,
        stream=False,
        max_new_tokens=200)
print(response.choices[0].message)


Model decided to use tool: 
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_exlvhtAlutM2Ueegg89724lU', function=Function(arguments='{\n  "search_query": "PM Lee Hsien Loong 2023 national day rally location"\n}', name='gg'), type='function')])
Model decided not to use tool: 
ChatCompletionMessage(content='Once upon a time, in a small town nestled between a sparkling river and a towering mountain, lived a young girl named Lily. She was known throughout the town for her radiant smile and immense kindness. One day, she discovered a magical stone that could grant one wish to the one who found it. Instead of wishing for wealth or fame, Lily wished for the happiness of everyone in her town. The next day, the town was filled with joyous laughter and cheer, everyone was happy. That', role='assistant', function_call=None, tool_calls=None)


In the following example, we demonstrate the same feature applied to Mistral-7B.

In [4]:
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('mistral7b-exllama')

print("Model decided to use tool: ")
user_prompt = "Where did PM Lee Hsien Loong hold his 2023 national day rally?"
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
        tools=tools,
        stream=False,
        max_new_tokens=200)
print(response.choices[0].message)

print("Model decided not to use tool: ")
user_prompt = "Tell me a one paragraph story."
response = gen.create(
    messages=[
        {'role':'user','content':user_prompt},
        {'role':'assistant','content':''}],
        tools=tools,
        stream=False,
        max_new_tokens=200)
print(response.choices[0].message)

Model decided to use tool: 
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_daf88dac-369c-498f-bd11-0801566a51df', function=Function(arguments='{"search_query": "PM Lee Hsien Loong 2023 National Day Rally location"}', name='gg'), type='function')])
Model decided not to use tool: 
ChatCompletionMessage(content=' Once upon a time, there was a little girl named Alice who lived in an enchanted forest full of talking animals and magical creatures. One day, she stumbled upon a secret door hidden behind a waterfall and decided to explore it. As she entered through the door, she found herself in a beautiful kingdom ruled by a kind queen. The queen welcomed her warmly and showed her around the castle, introducing her to all sorts of fascinating people and things. Alice had so much fun that she never wanted to leave!', role='assistant', function_call=None, tool_calls=None)


## 1.4 Running as a Service

### Step 1: Start Docker container

In [ ]:
%%bash

# Stop any container with the same name
docker rm -f gai-ttt

# Start the container
docker run -d \
    --name gai-ttt \
    -p 12031:12031 \
    --gpus all \
    -v ~/gai/models:/app/models \
    kakkoii1337/gai-ttt:latest

# Wait for model to load
sleep 30

# Confirm its running
docker logs gai-ttt

When the loading is completed, the logs should show this:

```bash
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:12031 (Press CTRL+C to quit)
```

### Step 2: Run Text Generation Client

The default model is Mistral7B-8k context size

In [ ]:
import requests,json
response = requests.post(
    url='http://localhost:12031/gen/v1/chat/completions', 
    json={
        "model": "mistral7b-exllama",
        "messages": [
            {"role": "user", "content": "Tell me a one paragraph short story."},
            {"role": "assistant", "content": ""}
        ],
        "max_new_tokens": 100,
        "stream": True
    },
    stream=True)
for chunk in response.iter_lines():
    result = json.loads(chunk.decode('utf-8'))
    print(result["choices"][0]["delta"]["content"],end='',flush=True)


In [16]:
import requests,json
response = requests.post(
    url='http://localhost:12031/gen/v1/chat/completions', 
    json={
        "model": "mistral7b-exllama",
        "messages": [
            {"role": "user", "content": "What is today's date?"},
            {"role": "assistant", "content": ""}
        ],
        "max_new_tokens": 100,
        "stream": False,
        "tools": [
        {
            "type": "function",
            "function": {
                "name": "gg",
                "description": "The 'gg' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }]
    },
    stream=False)

print(response.text)


{"id":"chatcmpl-53e917e4-2703-4587-bede-5a5de96f12f3","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_fb4edf0a-a509-4c24-91a3-78dd8b4fdcf5","function":{"arguments":"{\"search_query\": \"current date\"}","name":"gg"},"type":"function"}]}}],"created":1707944860,"model":"Mistral7B-ExLlama","object":"chat.completion","system_fingerprint":null,"usage":{"completion_tokens":0,"prompt_tokens":3620,"total_tokens":3620}}
